In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import joblib

# 生成示例训练数据（请替换为实际数据）
# 假设每个样本有9个特征（RGB, HSI, LAB），目标为水分值（0～100）
N = 1000  # 样本数量
X_train = np.random.randn(N, 9)   # 随机生成的9维特征数据
y_train = np.random.randn(N) * 10 + 50  # 模拟水分值，均值50，标准差10
y_train = np.clip(y_train, 0, 100)  # 限制水分值在0～100范围内

# 初始化SVR模型（参数可根据实际情况调节）
svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# 训练模型
svr.fit(X_train, y_train)

# 计算训练集均方误差（MSE）
pred_train = svr.predict(X_train)
mse = mean_squared_error(y_train, pred_train)
print(f"训练均方误差(MSE): {mse:.4f}")

# 保存训练好的模型到磁盘
model_path = 'moisture_svr_model.pkl'
joblib.dump(svr, model_path)
print("模型已保存到:", model_path)


训练均方误差(MSE): 101.8394
模型已保存到: moisture_svr_model.pkl


In [2]:
import cv2
import numpy as np
from skimage import color
import matplotlib.pyplot as plt
import joblib

# 加载保存的SVR模型
svr_model = joblib.load('moisture_svr_model.pkl')

# 读取图像，并转换为RGB格式
img = cv2.imread(r'C:\Users\You\Desktop\11.25\WPS1\3.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# 将图像归一化到0~1范围
img_norm = img.astype(np.float32) / 255.0
rows, cols, _ = img.shape

# --------------------
# 提取每个像素的9维特征
# --------------------

# 1. RGB特征
r = img_norm[:, :, 0]
g = img_norm[:, :, 1]
b = img_norm[:, :, 2]

# 2. HSI特征计算
# 计算色调（H）
num = 0.5 * ((r - g) + (r - b))
den = np.sqrt((r - g)**2 + (r - b) * (g - b)) + 1e-6  # 避免除零
theta = np.arccos(num / den)
H = theta.copy()
H[b > g] = 2 * np.pi - H[b > g]
H = H / (2 * np.pi)  # 归一化到[0,1]

# 计算饱和度（S）
S = 1 - 3 * np.minimum(np.minimum(r, g), b) / (r + g + b + 1e-6)

# 计算亮度（I）
I = (r + g + b) / 3

# 3. LAB特征转换（使用skimage库）
lab_img = color.rgb2lab(img_norm)
Lab_l = lab_img[:, :, 0]
Lab_a = lab_img[:, :, 1]
Lab_b = lab_img[:, :, 2]

# 将所有特征堆叠起来，形成形状为 (rows, cols, 9) 的特征矩阵
features = np.stack([r, g, b, H, S, I, Lab_l, Lab_a, Lab_b], axis=2)
# 将每个像素作为一个样本，转换为 (rows*cols, 9)
features_reshaped = features.reshape(-1, 9)

# --------------------
# 使用SVR模型对每个像素进行水分预测
# --------------------
moisture_values = svr_model.predict(features_reshaped)
# 将预测结果重新调整为图像大小，并限定在0～100范围内
moisture_map = moisture_values.reshape(rows, cols)
moisture_map = np.clip(moisture_map, 0, 100)

# --------------------
# 可视化原始图像和水分热图
# --------------------
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title('原始图像')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(moisture_map, cmap='jet')
plt.colorbar(label='水分含量')
plt.title('水分热图')
plt.axis('off')

plt.show()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
